In [ ]:
%pip install requests
%pip install rsa

In [ ]:
import requests
import rsa
import json
import binascii
import json
import random

G_BaseURL = "http://192.168.80.147/TESTwebservices_SWC"
#G_BaseURL = "http://78.158.168.230/TESTwebservices_SWC"
G_UserName = "admin"
G_PassWord = "admin"


def hex_string_to_bytes(hex_string):
    return binascii.unhexlify(hex_string)
def bytes_to_hex_string(byte_array):
    return binascii.hexlify(byte_array).decode()
def login(user_name=G_UserName, password=G_PassWord):
    url = G_BaseURL + "/Services/Framework/AuthenticationService.svc"
    session_url = url + "/session"
    login_url = url + "/login"
    response = requests.get(session_url)
    if response.status_code != 200:
        raise Exception("GET /session {}".format(response.status_code))
    session = json.loads(response.text)
    m = hex_string_to_bytes(session["rsa"]["M"])
    e = hex_string_to_bytes(session["rsa"]["E"])
    rsa_key = rsa.PublicKey(
        int.from_bytes(m, byteorder="big"), int.from_bytes(e, byteorder="big")
    )
    session_plus_password = session["id"] + "**" + password
    encrypted_password = rsa.encrypt(session_plus_password.encode(), rsa_key)
    headers = {"content-Type": "application/json"}
    data = {
        "sessionId": session["id"],
        "username": user_name,
        "password": bytes_to_hex_string(encrypted_password),
    }
    response = requests.post(login_url, headers=headers, data=json.dumps(data))
    if response.status_code != 200:
        raise Exception("POST /login {}".format(response.status_code))
    # session = response.headers["Set-Cookie"].split(",")[2].split(";")[0].strip()
    session = response.headers["Set-Cookie"].split(",")[1].split(";")[0].strip()
    return session


session = login()
print(session)

In [ ]:
"""
    get all  GetRegionalDivisionList
"""



def GetRegionalDivisionList(session):
    url = (
        G_BaseURL
        + "/General/AddressManagement/Services/AddressManagementWebService.svc/GetRegionalDivisionList"
    )
    headers = {"Cookie": session, "content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception("GET /GetAllCostCenters {}".format(response.status_code))
    return response.json()


# Use the session_id and auth_cookie from the login function
# cost_centers = get_all_cost_centers(session)
# print(cost_centers)
# json_str = json.dumps(your_list)
# parsed = json.loads(cost_centers)

# Pretty print the parsed data
# print(json.dumps(parsed, indent=4))

print(
    json.dumps(
        json.loads(json.dumps(GetRegionalDivisionList(session))),
        indent=4,
        ensure_ascii=False,
    )
)
# """

In [ ]:
# """


def salesStatementRegister(session, data):
    url = G_BaseURL + "/System/BusinessRuleEngine/Service.svc/call"
    headers = {"Cookie": session, "content-Type": "application/json"}
    JsonData = {"Name": "SalesStatementRegister", "Parameters": json.dumps(data)}
    response = requests.post(url, headers=headers, data=json.dumps(JsonData))
    if response.status_code != 200:
        #raise Exception("salesStatementRegister".format(response.status_code))
        print("hi")
    return response.json()


SalesStatement = {
    "Number": "",
    "SupplierCode" : "37801",

    "Date": "05/12/2024 00:00:00",
    "SalesOfficeRef": "1",  # همیشه 1
    "SalesAreaRef": "1",  # همیشه 1
    "RecipientType": "4",  # نوع تحویل گیرنده
    "RecipientRef": 88,  # شناسه مشتری تحویل گیرنده یا موسسه حمل
    "SellerBrokerRef": "1",  # همیشه 1
   
    "DeliveryAddressType": "1",  # نوع نشانی همیشه مشتری  1 موسسه حمل 2
    "StatementType": "1",  # همیشه 1 نوع اعلامیه
    "DeliverySpace": "08/09/2028 00:00:00",
    "ContractNo": str(random.randint(1, 99999999)),  # شماره قرارداد
    "PlantRef": "1",  # شماره مرکز همیشه 1
  
    "ReferenceType": "1",  # ؟همیشه 1
  
    "SalesTypeRef": "1",  # نوع فروش همیشه 1
    "InventoryRef": "3",  # شناسه انبار همیشه 3
    "SalesStatementWageResults": [  # کارمزدها
        {
            "BrokerServiceRef": "1",  # کارمزد فروش بورسی
            "CalculationPrecedence": "0",  # همیشه 0
            "CurrencyRef": "1",  # همیشه 1 نوع ارز
            "Value": "2502",  # مبلغ کارمز
            "ValueSetByUser": True,  # همیشه درست
        },
        {
            "BrokerServiceRef": "2",  # کارمزد حق درج
            "CalculationPrecedence": "0",
            "CurrencyRef": "1",
            "Value": "300",
            "ValueSetByUser": True,
        },
    ],
    "SalesStatementItems": [  # اقلام
        {
            "ProductRef": "9216",  # شناسه محصول همیشه 9216؟
            "ProductUnitRef": "1",  # واحد سنجش همیشه 1
            "InitialQuantity": "10",  # مقدار
            "Fee": "900000",  # نرخ
        }
    ],
}
# data = {"SalesStatement": SalesStatement}
# ERA = salesStatementRegister(session, {"SalesStatement": SalesStatement})
print(salesStatementRegister(session, {"SalesStatement": SalesStatement}))